In [78]:
library(dplyr)
library(fuzzyjoin)
library(readxl)
library(IRanges)

#install.packages("stringr")
library(stringr)
library(tidyr)

In [223]:
#samples <- snakemake@params[["samples"]]
#all_types <- snakemake@params[["all_types"]]


samples <- c('Row-01', 'Row-02', 'Row-03', 'Row-04',
             'Col-01', 'Col-02', 'Col-03', 'Col-04')

all_types <- c("Mu1_L", "Mu1_R",
"Mu2_L",
"Mu2_R",
"Mu3_L",
"Mu3_R",
"Mu4_L",
"Mu4_R",
"Mu5_L",
"Mu5_R",
"Mu7_L",
"Mu7_R",
"Mu8_L",
"Mu8_R",
"MuDR_L",
"MuDR_R",
"Mu10_L",
"Mu10_R",
"Mu11_L",
"Mu11_R",
"Mu12_L",
"Mu12_R",
"Mu13_L",
"Mu13_R",
"Mu14_L",
"Mu14_R",
"Mu15_L",
"Mu15_R",
"Mu16_L",
"Mu16_R",
"Mu17_L",
"Mu17_R",
"Mu18_L",
"Mu18_R",
"Mu19_L",
"Mu19_R")



all_insertions <- read.csv("results/insertions_table_final/all_identified_insertions.csv", header = TRUE)

In [224]:
base_all_types <- str_remove(all_types, c("_L"))
base_all_types <- str_remove(base_all_types, c("_R"))
base_all_types <- unique(base_all_types)
base_all_types

[1] "Mu1"  "Mu2"  "Mu3"  "Mu4"  "Mu5"  "Mu7"  "Mu8"  "MuDR" "Mu10" "Mu11"
[11] "Mu12" "Mu13" "Mu14" "Mu15" "Mu16" "Mu17" "Mu18" "Mu19"

In [225]:
#create insertions table with additional columns per (Mu) element/type
all_insertions_typed <- all_insertions

all_insertions_typed[,all_types]=0
#add additional column for uncategorized reads
all_insertions_typed[,"uncategorized"]=0

head(all_insertions_typed)

#for (t in all_types) {
#  all_insertions_typed %>%
#  tibble::add_column(get(t) = 0) %>%
#  head() %>% print()
#}

,Chr,Start,End,Sample,StartReads,EndReads,Mu1_L,Mu1_R,Mu2_L,Mu2_R,⋯,Mu15_R,Mu16_L,Mu16_R,Mu17_L,Mu17_R,Mu18_L,Mu18_R,Mu19_L,Mu19_R,uncategorized
,<chr>,<int>,<int>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,3124440,3124448,Col-01,2,2,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,1,3772030,3772038,Col-01,2,2,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
3,1,4498708,4498716,Col-01,8,5,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
4,1,4736284,4736292,Col-01,4,2,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
5,1,4909718,4909726,Col-01,9,8,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
6,1,8308462,8308470,Col-01,12,2,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [226]:
#read-in final sample files
for (s in samples) {
  tmp_type_file <- read.csv(paste0("results/te_typing/", s, "/", s, "_te_types_merged.tsv"), header = TRUE, sep="\t")
  tmp_type_file$Strand <- as.character(tmp_type_file$Strand)
  assign(paste0(s, "_type_file"), tmp_type_file)
}

head(`Col-02_type_file`)

,Name,Strand,Type
,<chr>,<chr>,<chr>
1,E00591:404:HFCMWCCX2:7:1101:10480:1573,1,Mu11_L
2,E00591:404:HFCMWCCX2:7:1101:16092:1625,1,Mu11_L
3,E00591:404:HFCMWCCX2:7:1101:13646:1678,1,Mu11_L
4,E00591:404:HFCMWCCX2:7:1101:14681:1678,1,Mu11_L
5,E00591:404:HFCMWCCX2:7:1101:13332:1766,1,Mu11_L
6,E00591:404:HFCMWCCX2:7:1101:18182:1766,1,Mu11_L


In [227]:
#modified, but core idea idea from here - https://stackoverflow.com/a/12088263
number2binary = function(number, noBits) {
       binary_vector = rev(as.numeric(intToBits(number)))
       if(missing(noBits)) {
          return(rev(binary_vector))
       } else {
          return(rev(binary_vector[-(1:(length(binary_vector) - noBits))]))
       }
}

#num=163
#number2binary(num, 8)[7]
#number2binary(num, 8)[8]

In [228]:
#need to load all sam files
for (s in samples) {
    SAM_path <- paste0("results/dedup_sam/", s, ".dedup.sam")
    SAM_object <- paste0("sam_", s)
    assign(SAM_object, read.csv(SAM_path, sep = "\t", header = FALSE, row.names = NULL))
    #take only the forst for columns and the sequence
    assign(SAM_object, get(SAM_object)[,c(1:4,10)])
    assign(SAM_object,
           get(SAM_object) %>%
             #some renaming of the header
             dplyr::rename(Name=V1, Flag=V2, Chr=V3, Start=V4) %>%
             #compute lengths of the alignments
             mutate(Length = str_length(V10)) %>%
             #compute and add End coordinate of reads
             mutate(End = Start+Length-1) %>%
             #drop Lengh & V10 (Sequence) columns
             select(-V10, -Length) %>%
             #translate Flag into forward (+) or reverse (-) strand mapping/read
             rowwise() %>%
             mutate(
               Strand = case_when(
                 number2binary(Flag, 8)[7] == 1 & number2binary(Flag, 8)[8] == 0 ~ "1",
                 number2binary(Flag, 8)[7] == 0 & number2binary(Flag, 8)[8] == 1 ~ "2",
                 TRUE ~ "LOST"
               )
             ) %>%
            select(-Flag) %>%
            relocate(Strand, .after=Name)
    )
}

In [135]:
head(`sam_Col-01`)

Name,Strand,Chr,Start,End
<chr>,<chr>,<chr>,<int>,<dbl>
E00591:404:HFCMWCCX2:7:1101:19197:55666,1,1,44317,44343
E00591:404:HFCMWCCX2:7:1101:19197:55666,2,1,44317,44343
E00591:404:HFCMWCCX2:7:1101:15138:9572,2,1,262584,262627
E00591:404:HFCMWCCX2:7:1101:15138:9572,1,1,262584,262627
E00591:404:HFCMWCCX2:7:1101:7811:72614,1,1,971328,971418
E00591:404:HFCMWCCX2:7:1101:7811:72614,2,1,971342,971418


### Annotating with type information

In [229]:
for (row in 1:nrow(all_insertions_typed[1:100,])) {
  
  #take row to work on
  tmp_ins <- all_insertions_typed[row,]

  #match Sample with sam object
  tmp_sam <- get(paste0("sam_", tmp_ins$Sample))

  #match sample of insertion file with te typing file
  tmp_type_file <- get(paste0(tmp_ins$Sample, "_type_file"))


  # fuzzyjoin for overlap + end or start needs to match
  tmp_merge_ins_sam <- fuzzyjoin::genome_inner_join(tmp_ins,
                             tmp_sam, 
                              by=c("Chr", "Start", "End")
                            ) %>%
                       select(Name, Strand, Chr.y, Start.y, End.y) %>%
                       dplyr::rename(Chr=Chr.y, Start=Start.y, End=End.y) %>%
                       filter(Start == tmp_ins$Start | 
                              Start == tmp_ins$End | 
                              End == tmp_ins$Start | 
                              End == tmp_ins$End)

  #associate sam reads with type information
  tmp_merge_ins_sam_typed_summary <- dplyr::left_join(tmp_merge_ins_sam, tmp_type_file, by = c("Name"="Name", "Strand"="Strand"), na_matches = "na") %>%
  group_by(Type) %>%
  summarize(n = n()) %>%
  #change NA to uncategorized
  tidyr::replace_na(list(Type = "uncategorized"))

  #add number of type association to current row
  for (t in tmp_merge_ins_sam_typed_summary$Type) {
    tmp_ins <- tmp_ins %>%
    mutate({{t}} := tmp_merge_ins_sam_typed_summary %>%
                 filter(Type == {{t}}) %>%
                 pull(n)
          )
  }
    
  #switch out new for old row in table
  all_insertions_typed[row,] <- tmp_ins
}




In [153]:
#all_insertions_typed %>%
#  filter(MuDR_L != 0) %>%
#  select(MuDR_L, MuDR_R, Mu8_L, Mu8_R)

#all_insertions_typed %>%
#  filter(Mu5_R != 0) %>%
#  select(Mu5_R, Mu7_L, Mu7_R)

### dev notes and tests

In [115]:
tmp_ins <- all_insertions_typed[3,]
#tmp_ins

#tmp_ins$Sample
#tmp_ins$Start
#tmp_ins$End


#match Sample with sam object
tmp_sam <- get(paste0("sam_", tmp_ins$Sample))

#match sample of insertion file with te typing file
tmp_type_file <- get(paste0(tmp_ins$Sample, "_type_file"))


# fuzzyjoin for overlap + end or start needs to match
tmp_merge_ins_sam <- fuzzyjoin::genome_inner_join(tmp_ins,
                             tmp_sam, 
                              by=c("Chr", "Start", "End")
                            ) %>%
select(Name, Strand, Chr.y, Start.y, End.y) %>%
#names()
dplyr::rename(Chr=Chr.y, Start=Start.y, End=End.y) %>%
filter(Start == tmp_ins$Start | 
       Start == tmp_ins$End | 
       End == tmp_ins$Start | 
       End == tmp_ins$End)


head(tmp_merge_ins_sam)

,Name,Strand,Chr,Start,End
,<chr>,<chr>,<chr>,<int>,<dbl>
1,E00591:404:HFCMWCCX2:7:1102:21947:14441,2,1,4498621,4498716
2,E00591:404:HFCMWCCX2:7:1101:2757:39036,1,1,4498662,4498716
3,E00591:404:HFCMWCCX2:7:1101:2757:39036,2,1,4498662,4498716
4,E00591:404:HFCMWCCX2:7:1101:20232:17061,1,1,4498667,4498716
5,E00591:404:HFCMWCCX2:7:1101:20232:17061,2,1,4498667,4498716
6,E00591:404:HFCMWCCX2:7:1101:23937:60255,2,1,4498708,4498766


In [39]:
#?genome_inner_join
#?findOverlaps

,Name,Strand,Type
,<chr>,<int>,<chr>
1,E00591:404:HFCMWCCX2:7:1101:17695:1731,1,Mu1_L
2,E00591:404:HFCMWCCX2:7:1101:12763:1836,1,Mu1_L
3,E00591:404:HFCMWCCX2:7:1101:16528:1854,1,Mu1_L
4,E00591:404:HFCMWCCX2:7:1101:19177:1872,1,Mu1_L
5,E00591:404:HFCMWCCX2:7:1101:20801:2645,1,Mu1_L
6,E00591:404:HFCMWCCX2:7:1101:6502:2803,1,Mu1_L


In [116]:
tmp_merge_ins_sam_typed_summary <- dplyr::left_join(tmp_merge_ins_sam, tmp_type_file, by = c("Name"="Name", "Strand"="Strand"), na_matches = "na") %>%
  group_by(Type) %>%
  summarize(n = n()) %>%
  #change NA to uncategorized
  tidyr::replace_na(list(Type = "uncategorized"))

for (t in tmp_merge_ins_sam_typed_summary$Type) {
  tmp_ins <- tmp_ins %>%
  mutate({{t}} := tmp_merge_ins_sam_typed_summary %>%
                 filter(Type == {{t}}) %>%
                 pull(n)
         )
}

Type,n
<chr>,<int>
Mu1_R,2
uncategorized,11


In [86]:
head(tmp_merge_ins_sam_typed_summary)

tmp_merge_ins_sam_typed_summary$Type

[1] "Mu1_R"         "uncategorized"

In [51]:
head(tmp_ins)

,Chr,Start,End,Sample,StartReads,EndReads,Mu1_L,Mu1_R,uncategorized
,<chr>,<int>,<int>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>
3,1,4498708,4498716,Col-01,8,5,0,0,0


In [117]:
for (t in tmp_merge_ins_sam_typed_summary$Type) {
  print(t)
}

[1] "Mu1_R"
[1] "uncategorized"


In [118]:
for (t in tmp_merge_ins_sam_typed_summary$Type) {
  tmp_ins <- tmp_ins %>%
  mutate({{t}} := tmp_merge_ins_sam_typed_summary %>%
                 filter(Type == {{t}}) %>%
                 pull(n)
         )
}

head(tmp_ins)

#tmp_ins %>%
#  mutate(Mu1_R = tmp_merge_ins_sam_typed_summary %>%
#                 filter(Type == "Mu1_R") %>%
#                 pull(n)
#         )

,Chr,Start,End,Sample,StartReads,EndReads,Mu1_L,Mu1_R,uncategorized
,<chr>,<int>,<int>,<chr>,<int>,<int>,<dbl>,<int>,<int>
3,1,4498708,4498716,Col-01,8,5,0,2,11


In [57]:
tmp_merge_ins_sam_typed_summary %>%
  filter(Type == "Mu1_R") %>%
  pull(n)

[1] 2

In [335]:
head(ttt)

,Mu1_total
,<dbl>
1,0
2,4
3,2
4,0
5,2
6,3


In [481]:
ttt <- all_insertions_typed[1:100,]
#head(ttt)


for (t in base_all_types) {
    
  tmp_left <- paste0(t, "_L")
  tmp_right <- paste0(t, "_R")
    
  ttt <- ttt %>%
#    rowwise() %>%
    #add total amount of reads supporting insertion
    mutate(TotalReads = StartReads + EndReads) %>%
    dplyr::relocate(TotalReads, .after = EndReads) %>%
    #add total amount of support reads per (Mu) type
    mutate("{t}_total" := .data[[tmp_left]] + .data[[tmp_right]]) #%>%
#    rowwise() %>%
#    select(., matches("_total")) #%>%
#    mutate(Class = names(c(StartReads, EndReads))[max.col(.)])
}

head(ttt)

#create df to write to
sss <- ttt %>%
  select(., matches(c("_total", "uncategorized")))
sss[,c("max_value","count")]=0

#loop over df to identify uncear cases in which the max support for type classification is tied
for (l in 1:nrow(ttt)) {
tmp_tie_check <- ttt[l,] %>%
  select(., matches(c("_total", "uncategorized"))) %>%
  rowwise() %>%
  mutate(max_value = max(across())) %>%
#  mutate(across(.fns = ~ as.integer(.x))) %>%
  rowwise() %>%
  mutate(count = length(which(.==max_value)))
    
  sss[l,] <- tmp_tie_check
}

head(sss)

sss %>%
  mutate(Clear = 
    case_when(
      count > 2 ~ "TIED",
      count == 2 ~ "Clear",
      TRUE ~ "ERROR"
    )
  ) %>% head()

#  mutate(max = names(.)[max.col(., ties.method = "first")]) %>%
#  mutate(sum_yes = rowSums(. == max_value)) %>%


,Chr,Start,End,Sample,StartReads,EndReads,TotalReads,Mu1_L,Mu1_R,Mu2_L,⋯,Mu10_total,Mu11_total,Mu12_total,Mu13_total,Mu14_total,Mu15_total,Mu16_total,Mu17_total,Mu18_total,Mu19_total
,<chr>,<int>,<int>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,3124440,3124448,Col-01,2,2,4,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,1,3772030,3772038,Col-01,2,2,4,2,2,0,⋯,0,0,0,0,0,0,0,4,0,0
3,1,4498708,4498716,Col-01,8,5,13,0,2,0,⋯,0,2,0,0,2,0,2,4,0,2
4,1,4736284,4736292,Col-01,4,2,6,0,0,0,⋯,0,1,0,0,1,0,1,0,0,1
5,1,4909718,4909726,Col-01,9,8,17,0,2,0,⋯,0,0,0,0,0,0,0,4,0,0
6,1,8308462,8308470,Col-01,12,2,14,1,2,0,⋯,0,0,0,0,0,0,0,4,0,0


,Mu1_total,Mu2_total,Mu3_total,Mu4_total,Mu5_total,Mu7_total,Mu8_total,MuDR_total,Mu10_total,Mu11_total,⋯,Mu13_total,Mu14_total,Mu15_total,Mu16_total,Mu17_total,Mu18_total,Mu19_total,uncategorized,max_value,count
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,4,4,2
2,4,2,0,0,2,4,0,0,0,0,⋯,0,0,0,0,4,0,0,0,4,4
3,2,0,0,0,2,4,3,6,0,2,⋯,0,2,0,2,4,0,2,6,6,3
4,0,0,0,0,0,0,2,4,0,1,⋯,0,1,0,1,0,0,1,3,4,2
5,2,0,4,8,2,4,0,0,0,0,⋯,0,0,0,0,4,0,0,11,11,2
6,3,1,2,4,2,4,0,0,0,0,⋯,0,0,0,0,4,0,0,9,9,2


,Mu1_total,Mu2_total,Mu3_total,Mu4_total,Mu5_total,Mu7_total,Mu8_total,MuDR_total,Mu10_total,Mu11_total,⋯,Mu14_total,Mu15_total,Mu16_total,Mu17_total,Mu18_total,Mu19_total,uncategorized,max_value,count,Clear
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,4,4,2,Clear
2,4,2,0,0,2,4,0,0,0,0,⋯,0,0,0,4,0,0,0,4,4,TIED
3,2,0,0,0,2,4,3,6,0,2,⋯,2,0,2,4,0,2,6,6,3,TIED
4,0,0,0,0,0,0,2,4,0,1,⋯,1,0,1,0,0,1,3,4,2,Clear
5,2,0,4,8,2,4,0,0,0,0,⋯,0,0,0,4,0,0,11,11,2,Clear
6,3,1,2,4,2,4,0,0,0,0,⋯,0,0,0,4,0,0,9,9,2,Clear


In [479]:
head(tmp_tie_check)
head(sss)

Mu1_total,Mu2_total,Mu3_total,Mu4_total,Mu5_total,Mu7_total,Mu8_total,MuDR_total,Mu10_total,Mu11_total,⋯,Mu13_total,Mu14_total,Mu15_total,Mu16_total,Mu17_total,Mu18_total,Mu19_total,uncategorized,max_value,count
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
0,0,1,2,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,7,7,2


,Mu1_total,Mu2_total,Mu3_total,Mu4_total,Mu5_total,Mu7_total,Mu8_total,MuDR_total,Mu10_total,Mu11_total,Mu12_total,Mu13_total,Mu14_total,Mu15_total,Mu16_total,Mu17_total,Mu18_total,Mu19_total,uncategorized,max_value
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,4
2,4,2,0,0,2,4,0,0,0,0,0,0,0,0,0,4,0,0,0,4
3,2,0,0,0,2,4,3,6,0,2,0,0,2,0,2,4,0,2,6,6
4,0,0,0,0,0,0,2,4,0,1,0,0,1,0,1,0,0,1,3,4
5,2,0,4,8,2,4,0,0,0,0,0,0,0,0,0,4,0,0,11,11
6,3,1,2,4,2,4,0,0,0,0,0,0,0,0,0,4,0,0,9,9


In [320]:
ttt <- all_insertions_typed[1:100,]

head(ttt, n=1)

ttt %>%
  select(., ends_with("_total")) %>%
#  rowwise() %>%
#  mutate(row_max = names(.)[which.max(c("StartReads","EndReads"))]) %>%
#  relocate(row_max, .after = uncategorized) %>%
  head()

,Chr,Start,End,Sample,StartReads,EndReads,Mu1_L,Mu1_R,Mu2_L,Mu2_R,⋯,Mu15_R,Mu16_L,Mu16_R,Mu17_L,Mu17_R,Mu18_L,Mu18_R,Mu19_L,Mu19_R,uncategorized
,<chr>,<int>,<int>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,3124440,3124448,Col-01,2,2,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,4


1
2
3
4
5
6
